In [1]:
from IPython.display import clear_output
!pip install git+https://github.com/shijianjian/EfficientNet-PyTorch-3D
clear_output()

In [2]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=70ea51f630928d7e563794d618b92b788d7efcd3b2234f02b0a2c22a12fa84ee
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [3]:
pip show pydicom

Name: pydicom
Version: 2.4.4
Summary: A pure Python package for reading and writing DICOM data
Home-page: 
Author: 
Author-email: Darcy Mason and contributors <darcymason@gmail.com>
License: 
Location: /opt/conda/lib/python3.10/site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os

# Specify your dataset directory
dataset_dir = "/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification"
# Get a list of all file names in the dataset directory
file_names = [f for f in os.listdir(dataset_dir) if os.path.isfile(os.path.join(dataset_dir, f))]

print(file_names)

['sample_submission.csv', 'train_labels.csv']


# Let's start with the first step: Data Preprocessing. Here's how you can load DICOM images from each patient directory and preprocess them using Python with the PyDICOM and OpenCV libraries:

In [5]:
import os
import cv2
import numpy as np
import pydicom
import glob

def load_dicom_images(patient_dir):
    """
    Load DICOM images from a patient directory.
    
    Parameters:
    - patient_dir: Path to the directory containing DICOM images.
    
    Returns:
    - images: List of loaded DICOM images.
    """
    images = []
    for filepath in glob.glob(os.path.join(patient_dir, '**', '*.dcm'), recursive=True):
        try:
            ds = pydicom.dcmread(filepath)
            img = ds.pixel_array
            images.append(img)
        except Exception as e:
            pass
    return images

def preprocess_images(images):
    """
    Preprocess a list of images.
​
    Parameters:
    - images: List of input images.
​
    Returns:
    - preprocessed_images: List of preprocessed images.
    """
    preprocessed_images = []
    for img in images:
        # Resize image to desired dimensions (e.g., 256x256)
        img = cv2.resize(img, (256, 256))
        # Normalize pixel values (if needed)
        img = img / 255.0
        preprocessed_images.append(img)
    return preprocessed_images

## Example usage:
dataset_dir = "/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/"
print("Files in dataset directory:", os.listdir(dataset_dir)[:5])  # Corrected line

# Debug print: Print first 5 characters of DICOM file paths
count = 0
for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        if file.endswith(".dcm"):
            print(os.path.join(root, file))
            count += 1
            if count >= 5:
                break
    if count >= 5:
        break
        
images = load_dicom_images(dataset_dir)
print("Number of DICOM images loaded:", len(images))
preprocessed_images = preprocess_images(images)
print("Number of preprocessed images:", len(preprocessed_images))


Files in dataset directory: ['Image-273.dcm', 'Image-245.dcm', 'Image-365.dcm', 'Image-130.dcm', 'Image-98.dcm']
/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/Image-273.dcm
/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/Image-245.dcm
/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/Image-365.dcm
/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/Image-130.dcm
/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/Image-98.dcm
Number of DICOM images loaded: 400
Number of preprocessed images: 400


In [7]:
import os
num_files = sum([len(files) for r, d, files in os.walk("/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/")])
print(f"Total number of DICOM files: {num_files}")


Total number of DICOM files: 400116


In [8]:
import os

total_size = 0
num_files = 0
for dirpath, dirnames, filenames in os.walk("/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/"):
    for f in filenames:
        fp = os.path.join(dirpath, f)
        total_size += os.path.getsize(fp)
        num_files += 1
avg_size_mb = (total_size / num_files) / (1024 * 1024)  # Convert bytes to MB
print(f"Average file size: {avg_size_mb:.2f} MB")


Average file size: 0.33 MB


In [1]:
import psutil
import os
import subprocess

print("CPU Info:", os.cpu_count())
print("RAM Info:", psutil.virtual_memory())
gpu_info = subprocess.check_output(['nvidia-smi']).decode('utf-8')
print("GPU Info:", gpu_info)



CPU Info: 4
RAM Info: svmem(total=33669926912, available=32513171456, percent=3.4, used=678666240, free=26040758272, active=1685721088, inactive=5176193024, buffers=1380655104, cached=5569847296, shared=999424, slab=549867520)
GPU Info: Thu May 16 09:57:42 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C

In [3]:
import time
import pydicom
import cv2

start_time = time.time()
ds = pydicom.dcmread('/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/Image-1.dcm')  # Provide a sample DICOM file path
img = ds.pixel_array
img = cv2.resize(img, (256, 256))
img = img / 255.0
end_time = time.time()
print(f"Time taken to preprocess one image: {end_time - start_time:.2f} seconds")


Time taken to preprocess one image: 0.04 seconds


In [4]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img = np.expand_dims(img, axis=-1)  # Use the preprocessed image from the previous step
batch_img = np.expand_dims(img, axis=0)  # Create a batch with one image
augmenter = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)
start_time = time.time()
augmented_img = next(augmenter.flow(batch_img, batch_size=1))
end_time = time.time()
print(f"Time taken to augment one image: {end_time - start_time:.2f} seconds")


2024-05-16 09:58:11.744612: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 09:58:11.744707: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 09:58:11.873532: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Time taken to augment one image: 0.03 seconds


In [ ]:
import os
import numpy as np
import pydicom
import cv2
import glob
import multiprocessing

def load_dicom_images(patient_dir):
    """
    Load DICOM images from a patient directory.
    
    Parameters:
    - patient_dir: Path to the directory containing DICOM images.
    
    Returns:
    - images: List of loaded DICOM images.
    """
    images = []
    for filepath in glob.glob(os.path.join(patient_dir, '**', '*.dcm'), recursive=True):
        try:
            ds = pydicom.dcmread(filepath)
            img = ds.pixel_array
            images.append(img)
        except Exception as e:
            pass
    return images

def preprocess_images(images):
    """
    Preprocess a list of images.
    
    Parameters:
    - images: List of input images.
    
    Returns:
    - preprocessed_images: List of preprocessed images.
    """
    preprocessed_images = []
    for img in images:
        # Resize image to desired dimensions (e.g., 256x256)
        img = cv2.resize(img, (256, 256))
        # Normalize pixel values
        img = img / 255.0
        preprocessed_images.append(img)
    return preprocessed_images

# Define the dataset directory
dataset_dir = "/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/"

# Load DICOM images from 100 patient directories using multiprocessing
patient_dirs = [os.path.join(dataset_dir, dir) for dir in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, dir))][:100]

pool = multiprocessing.Pool(processes=4)  # Adjust the number of processes as needed
images_list = pool.map(load_dicom_images, patient_dirs)
pool.close()
pool.join()

# Flatten the list of lists into a single list of images
images = [img for sublist in images_list for img in sublist]

# Preprocess the images
preprocessed_images = preprocess_images(images)


# The second step is Data Augmentation. Data augmentation is crucial for training robust machine learning models, especially when dealing with limited datasets. Here's how you can augment your preprocessed images using the ImageDataGenerator class from Keras

import os
import cv2
import numpy as np
import pydicom
import glob
import pandas as pd
from multiprocessing import Pool
from functools import partial
from tensorflow.keras.preprocessing.image import ImageDataGenerator


def load_labels(file_path):
    """
    Load MGMT values from the train_labels.csv file into a dictionary.
    
    Parameters:
    - file_path: Path to the train_labels.csv file.
    
    Returns:
    - labels_dict: Dictionary mapping patient IDs to MGMT values.
    """
    labels_df = pd.read_csv(file_path)
    labels_dict = dict(zip(labels_df['BraTS21ID'], labels_df['MGMT_value']))
    return labels_dict

def load_dicom(filepath):
    """
    Load and preprocess a single DICOM image.
    
    Parameters:
    - filepath: Path to the DICOM file.
    
    Returns:
    - img: Preprocessed image.
    """
    try:
        ds = pydicom.dcmread(filepath)
        img = ds.pixel_array
        img = cv2.resize(img, (256, 256))
        img = img / 255.0
        return img
    except Exception as e:
        return None

def load_dicom_batch(filepaths):
    """
    Load and preprocess a batch of DICOM images in parallel.
    
    Parameters:
    - filepaths: List of file paths to DICOM files.
    
    Returns:
    - batch_images: Batch of preprocessed images.
    """
    with Pool() as pool:
        batch_images = pool.map(load_dicom, filepaths)
    return np.array(batch_images)

def load_dicom_images_generator(dataset_dir, labels_dict, batch_size=256):
    """
    Generator function to load DICOM images from patient directories in batches.
    
    Parameters:
    - dataset_dir: Path to the directory containing patient directories.
    - labels_dict: Dictionary mapping patient IDs to MGMT values.
    - batch_size: Number of images to load per batch.
    
    Yields:
    - batch_images: Batch of preprocessed images.
    - batch_labels: Batch of MGMT labels.
    """
    for patient_dir in os.listdir(dataset_dir):
        patient_path = os.path.join(dataset_dir, patient_dir)
        if os.path.isdir(patient_path):
            filepaths = glob.glob(os.path.join(patient_path, '**', '*.dcm'), recursive=True)
            filepaths = filepaths[:batch_size]  # Select only first batch_size files
            batch_images = load_dicom_batch(filepaths)
            patient_id = os.path.basename(patient_path)
            mgmt_value = labels_dict.get(patient_id, -1)  # Default value of -1 if not found
            yield batch_images, mgmt_value

# Data augmentation using Keras ImageDataGenerator
data_augmenter = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# Example usage:
dataset_dir = "/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/"
labels_file = "/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv"

# Load MGMT labels
labels_dict = load_labels(labels_file)

# Load DICOM images and labels using the generator
image_label_generator = load_dicom_images_generator(dataset_dir, labels_dict)

# Process images and labels in batches with data augmentation
for batch_index, (batch_images, batch_labels) in enumerate(image_label_generator):
    # Reshape batch_images to have rank 4
    batch_images = np.expand_dims(batch_images, axis=-1)  # Add channel dimension
    batch_images = np.repeat(batch_images, 3, axis=-1)  # Repeat grayscale image to 3 channels
    augmented_images = next(data_augmenter.flow(batch_images, batch_size=256, shuffle=False))
    print(f"Batch {batch_index + 1}: Augmented images - {augmented_images.shape}, Labels - {batch_labels}")



# 1. Setup and Imports

In [10]:
import os
import cv2
import numpy as np
import pydicom
import pandas as pd
import glob
import logging
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Ensure you run this segment first


# 2. Logging Configuration

In [11]:
# Set up logging
logging.basicConfig(filename='progress.log', level=logging.INFO)

def log_progress(batch_index, batch_images, batch_labels):
    logging.info(f"Batch {batch_index + 1}: Processed {len(batch_images)} images with labels {batch_labels}")
    if batch_index % 10 == 0:  # Print progress every 10 batches
        print(f"Processed {batch_index + 1} batches.")

# Ensure you run this segment second


# 3. Generator Function Definition

In [12]:
def load_dicom_images_generator(dataset_dir, batch_size=32):
    """
    Generator function to load DICOM images from patient directories in batches.
    
    Parameters:
    - dataset_dir: Path to the directory containing patient directories.
    - batch_size: Number of images to load per batch.
    
    Yields:
    - batch_images: Batch of preprocessed images.
    - batch_labels: Batch of corresponding labels.
    """
    train_labels = pd.read_csv("/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")
    image_paths = []
    labels = []

    for patient_dir in os.listdir(dataset_dir):
        patient_path = os.path.join(dataset_dir, patient_dir)
        if os.path.isdir(patient_path):
            for filepath in glob.glob(os.path.join(patient_path, '*.dcm')):
                image_id = os.path.basename(filepath).split('-')[1].split('.')[0]
                label_data = train_labels[train_labels['BraTS21ID'] == int(image_id)]['MGMT_value']
                if not label_data.empty:
                    label = label_data.values[0]
                    labels.append(label)
                    image_paths.append(filepath)
    
    num_images = len(image_paths)
    indices = np.random.permutation(num_images)
    
    while True:
        for i in range(0, num_images, batch_size):
            batch_indices = indices[i:i+batch_size]
            batch_images = []
            batch_labels = []
            for idx in batch_indices:
                filepath = image_paths[idx]
                try:
                    ds = pydicom.dcmread(filepath)
                    img = ds.pixel_array
                    img = cv2.resize(img, (256, 256))
                    img = img / 255.0
                    batch_images.append(img)
                    batch_labels.append(labels[idx])
                except Exception as e:
                    logging.error(f"Error processing {filepath}: {e}")
            
            yield np.array(batch_images), np.array(batch_labels)

# Ensure you run this segment third


# 4. Data Augmentation Setup

In [13]:
# Data augmentation
data_augmenter = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# Ensure you run this segment fourth


# 5. Main Processing Loop with Checkpoints

In [ ]:
# Example usage
dataset_dir = "/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/"
batch_size = 32

# Load DICOM images and labels using the generator
image_label_generator = load_dicom_images_generator(dataset_dir, batch_size=batch_size)

# Ensure you run this segment fifth

import time

# Process images and labels in batches with data augmentation
for batch_index, (batch_images, batch_labels) in enumerate(image_label_generator):
    start_time = time.time()
    
    # Perform data augmentation
    augmented_generator = data_augmenter.flow(np.expand_dims(batch_images, axis=-1), batch_labels, batch_size=batch_size, shuffle=False)
    
    # Iterate over augmented batches
    for augmented_images, augmented_labels in augmented_generator:
        log_progress(batch_index, augmented_images, augmented_labels)
        break  # Stop after processing the first augmented batch in the generator

    end_time = time.time()
    elapsed_time = end_time - start_time
    
    # Print elapsed time for processing the batch
    print(f"Batch {batch_index + 1} processed in {elapsed_time:.2f} seconds.")
    
    # Checkpoint: Save progress every 10 batches
    if (batch_index + 1) % 10 == 0:
        print(f"Processed {batch_index + 1} batches.")

    # For testing purposes, limit to a few batches to observe initial output
    if batch_index >= 9:  # Change this number as needed for testing
        break


In [ ]:
import os
num_files = sum([len(files) for r, d, files in os.walk("/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/")])
print(f"Total number of DICOM files: {num_files}")
